# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Ignacio Guerra Sevillano  <br>
Url: <br>
Problema: 1. Sesiones de doblaje <br>



Descripción del problema:
Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible con los datos aportados en el fichero excel.



(*) La respuesta es obligatoria







Se trata de un problema que puede ser modelizado como un problema de Programación Lineal Binario. En nuestro caso tomaremos el número de días como un valor fijo. En concreto, lo fijaremos en 5 días, pues siempre será igual o mejor meter más tomas en el mismo día que empezar un día nuevo por una toma. 

Definimos las variables binarias $X_{ij}$ con $i\in \{1,\dots, 30\}$ y $j\in \{1,\dots, 5\}$ como $$X_{ij} = \quad \text{$1$ si la toma $i$ se realiza en el día $j$,} \quad \text{$0$ en caso contrario}.$$

Puesto que cada toma solo se realiza una vez (obvio), se tienen las siguientes restricciones:
$$\sum_{i=1}^{30} X_{ij} = 1, \quad j\in \{1,\dots, 5\}.$$

Si además tenemos en cuenta que no se puede hacer más de 6 tomas en el mismo día, tendremos que:
$$\sum_{j=1}^{5} X_{ij} = 6, \quad i\in \{1,\dots, 30\}.$$

Se da la igualdad pues para que sean 5 días hay que realizar exactamente 6 tomas por día.

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Las posibilidades sin tener en cuenta las restricciones pero sí suponiendo que cada toma se debe realizar tan solo una vez son exactamente $5^{30}$ (basta fijarse en la primera igualdad y escoger para cada una de las 5 ecuaciones una variables que tome el valor 1 de las 30). Esto último sería cierto en caso de que el número de días fuese fijo, si no fuese fijo las combinaciones posibles serían $30^{30}$ por el mismo razonamiento (suponiendo que como mínimo se hace una toma por día y los días son consecutivos).

Para el caso que nos corresponde, es decir, número de días igual a 5, las posibilidades con las restricciones son:

$$n_{pos} = \binom{30}{6} \cdot \binom{24}{6} \cdot \binom{18}{6} \cdot \binom{12}{6} \cdot \binom{6}{6} = 1370874167589326400 \approx 1.371 \cdot 10^{18}.$$

Lo último se debe a que para e primer día podemos escoger 6 tomas de las 30, para el segundo día tenemos que escoger 6 tomas de las restantes, es decir, 6 tomas de 24, para el tercero 6 tomas de 18 y así hasta el último día.

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Debido al formato excel de los datos de entrada y a que no se requiere apenas operaciones matemáticas para diseñar el algoritmo he utilizado la estructura de DataFrame de Pandas.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

La función objetivo a **minimizar** es $$f(X_{ij}) = \sum_{i=1}^{30} \sum_{j=1}^{5} \omega_i X_{ij},$$

donde los $\omega_i$ corresponden con el número de actores de la toma $i$.

(*)Diseña un algoritmo que mejore la complejidad del algoritmo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

En primer lugar, importamos los datos:

In [9]:
# Importamos librerías a utilizar
import pandas as pd
import random
import numpy as np


def importar_datos(archivo_excel):
    
    # Lee el archivo y lo convierte a DataFrame
    df = pd.read_excel(archivo_excel, skiprows=1)

    # Establece la primera columna como índices
    df = df.set_index(df.columns[0])
    
    # Eliminamos filas y columnas en blanco
    df.drop(['Unnamed: 11'], axis=1, inplace=True)
    df.drop(df.index[[30, 31]], axis=0, inplace=True)
    
    df = df.fillna(df['Total'].sum())
    df = df.astype(int)
    
    return df

# Muestra el dataframe resultante
importar_datos('datos.xlsx')

      1  2  3  4  5  6  7  8  9  10  Total
Toma                                      
1     1  1  1  1  1  0  0  0  0   0      5
2     0  0  1  1  1  0  0  0  0   0      3
3     0  1  0  0  1  0  1  0  0   0      3
4     1  1  0  0  0  0  1  1  0   0      4
5     0  1  0  1  0  0  0  1  0   0      3
6     1  1  0  1  1  0  0  0  0   0      4
7     1  1  0  1  1  0  0  0  0   0      4
8     1  1  0  0  0  1  0  0  0   0      3
9     1  1  0  1  0  0  0  0  0   0      3
10    1  1  0  0  0  1  0  0  1   0      4
11    1  1  1  0  1  0  0  1  0   0      5
12    1  1  1  1  0  1  0  0  0   0      5
13    1  0  0  1  1  0  0  0  0   0      3
14    1  0  1  0  0  1  0  0  0   0      3
15    1  1  0  0  0  0  1  0  0   0      3
16    0  0  0  1  0  0  0  0  0   1      2
17    1  0  1  0  0  0  0  0  0   0      2
18    0  0  1  0  0  1  0  0  0   0      2
19    1  0  1  0  0  0  0  0  0   0      2
20    1  0  1  1  1  0  0  0  0   0      4
21    0  0  0  0  0  1  0  1  0   0      2
22    1  1 

Se diseña una función que dada una toma del dataframe, devuelve las tomas con los mismos actores o menos, los actores de dicha toma y el número total de actores:

In [48]:
def tomas_cercanas(toma, df):
    
    tomas_finales = []
    
    # Actores de la toma de referencia
    actores = set(df.columns[df.loc[toma] == 1])
    
    # Bucle que recorre las tomas y añade las que tengan mismos 
    # actores o menos que la toma de referencia
    for toma_prueba in df.index:
        actores_prueba = set(df.columns[df.loc[toma_prueba] == 1])
        if actores_prueba <= actores:
            tomas_finales.append(toma_prueba)
    
    # Número de actores de la toma de referencia
    num_actores_maximo = len(actores)    
            
    return tomas_finales, actores, num_actores_maximo

Ahora se muestra el siguiente algoritmo voraz diseñado para este problema. En el primer paso se escoge una de las tomas con máximo o casi máximo número de actores (aquí introducimos un parámetro i de aleatoriedad con el fin de diversificar). A continuación, se va llenando ese día con tomas de forma que se escoge la toma que menos aumenta el coste, es decir, primero se cogen escenas que tengan los mismos actores o menos y luego aquellas que difieren con la toma de referencia en menos actores. Se continua así hasta llegar a 6 tomas, de forma que se cierra el día y se empieza de nuevo.

In [49]:
def algoritmo_voraz(data, i):
    
    df = data.copy()
    
    # Ordenamos el DataFrame por número de actores
    df.sort_values('Total', ascending=False, inplace=True)
    
    # Valores iniciales
    solucion = []
    tomas_restantes = list(df.index)
    coste = 0
    
    # Bucle que se ejecuta mientras queden tomas sin asignar a un día
    while len(tomas_restantes) > 0:
        
        # Se inicializa el día y se escoge una toma aleatoria de las
        # i primeras con más número de actores
        dia = []
        toma_actual = random.choice(tomas_restantes[:i]) #TOMA DE REFERENCIA
        
        # Las tomas locales son las tomas con mismos actores o menos,
        # las tomas no locales el resto de las que están sin asignar
        tomas_locales, actores, num_actores_toma = tomas_cercanas(toma_actual, df)
        tomas_no_locales = list(set(tomas_restantes) - set(tomas_locales))
        
        # El coste es el número de actores de la toma
        coste = coste + num_actores_toma
        
        # Bucle que va llenando tomas del día correspondiente
        while len(dia) < 6:
            
            # Primero se añaden tomas locales si hay
            if len(tomas_locales) > 0:
                dia.append(tomas_locales[0])
                tomas_locales = tomas_locales[1:]
                
            else:
                coste_aux = float('Inf')
                
                # Bucle que recorres las tomas no locales y se queda 
                # con que suma menor coste
                for toma in tomas_no_locales:
                    actores_toma = set(df.columns[df.loc[toma] == 1])
                    coste_toma = len(actores_toma - set(actores))
                    
                    if coste_toma <= coste_aux:
                        coste_aux = coste_toma
                        toma_aux = toma 
     
                dia.append(toma_aux)
                tomas_no_locales.remove(toma_aux)
                coste = coste + coste_aux 
        
        # Se añade a la solución el día con sus tomas correspondientes
        solucion.append(dia)
        
        # Se actualiza el DataFrame y las tomas restantes para la siguiente
        # iteración
        tomas_restantes = [toma for toma in tomas_restantes if toma not in dia]
        df = df.loc[tomas_restantes, :]
        
    return solucion, coste

Con el fin de buscar distintas soluciones y diversificar, se aplica el algoritmo anterior variando el valor i y repitiendo el proceso 20 veces, con el fin de encontrar una mejor solución.

In [61]:
def resolver(data):

    coste_final = float('Inf')
    
    # Se varía el parámetro de aleatoriedad i
    for i in range(1, 5):
        for _ in range(20):
            solucion, coste = algoritmo_voraz(data, i)

            if coste <= coste_final:
                solucion_final = solucion
                coste_final = coste
    
    # Se transforma la solución a DataFrame
    solucion_final = pd.DataFrame(solucion_final)       
    solucion_final.index = ['Día 1', 'Día 2', 'Día 3', 'Día 4', 'Día 5']
    solucion_final.columns = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6']

    return solucion_final, coste_final

In [67]:
plan, coste = resolver(df)

print(f'El coste final es {coste} y la planificación es:\n')
plan

El coste final es 38 y la planificación es:



,T1,T2,T3,T4,T5,T6
Día 1,1,22,20,7,6,9
Día 2,11,23,19,17,30,29
Día 3,12,8,14,24,28,18
Día 4,26,27,13,2,21,16
Día 5,4,15,5,3,10,25


(*)Calcula la complejidad del algoritmo

Sea $n$ el número de tomas totales, $m$ el número de actores totales y $a$ el número máximo de tomas que se pueden realizar en un día (en este caso $a=6$). Para calcular la complejidad del algoritmo tengamos en cuenta los siguientes puntos:
- La complejidad de .sort_values() es la de un algoritmo **timsort**, es decir, $n\log(n)$.
- La complejidad de la función auxiliar *vecinos_cercanos* es del orden $O(nm)$, pues consta de un bucle que recorre las tomas y en cada iteración hace $m$ comprobaciones (una por columna actor).
- La complejidad de la función final *resolver* es la misma que la de la función *algoritmo_voraz*, salvo constante.

Basta entonces hallar la complejidad de la función *algoritmo_voraz* para obtener la complejidad final. El bucle **while** principal ejecuta $n/a$ iteraciones, dicho cociente queda multiplicado por la suma de los siguientes términos:
- $(n-k)m$ operaciones de la función *vecinos_cercanos*, siendo $k$ el número de tomas que ya se han eliminado hasta dicha iteración de las tomas restantes.
- 3 operaciones por la suma del coste, la operación entre conjuntos y añadir el día a la solcuión final.
- $(n-k)$ operaciones para actualizar la lista de tomas restantes.
- El bucle **while** para añadir tomas al día toma $a$ iteraciones cada vez. En el mejor de los casos realiza $a$ operaciones dentro, mientras que en el peor de los casos (cuando nunca hay tomas con mismos o menos actores que la toma de referencia) se realiza $(n-k)m$ operaciones.

En conclusión, tendremos una complejidad de la forma $O(n\log(n)) + O(n^2m)$ tanto en el mejor de los casos como en el peor. Pues que $O(n\log(n))$ es despreciable frente a $O(n^2m)$, la complejidad de todo el algoritmo es del orden $O(n^2m)$

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

In [62]:
# Función para generar valores aleatorios
def generar_valor_aleatorio(_):
    return np.random.randint(2)

# Reemplazar todos los valores por valores aleatorios
df2 = df.applymap(generar_valor_aleatorio)
df2['Total'] = df2.iloc[:, 1:10].sum(axis=1)
print(df2)

      1  2  3  4  5  6  7  8  9  10  Total
Toma                                      
1     0  0  0  0  0  1  0  1  1   0      3
2     1  0  1  1  1  0  0  1  0   0      4
3     1  0  0  0  1  1  0  1  1   1      5
4     1  0  1  1  1  0  0  1  0   1      5
5     0  1  1  0  0  0  0  1  0   1      4
6     0  1  0  0  1  0  1  1  1   0      5
7     1  1  0  0  0  0  1  1  0   0      3
8     0  1  0  0  1  0  1  0  0   0      3
9     1  0  1  1  0  1  0  1  1   1      6
10    0  1  1  1  0  1  1  1  0   1      7
11    1  1  0  0  1  1  1  1  0   0      5
12    0  1  1  0  0  0  1  1  0   0      4
13    0  1  1  0  0  0  0  1  0   1      4
14    1  1  0  0  0  1  1  1  0   0      4
15    1  1  0  0  1  1  0  0  0   1      4
16    0  0  1  0  1  1  1  1  1   1      7
17    0  0  0  1  1  0  1  1  0   0      4
18    1  0  1  1  0  0  0  0  1   1      4
19    1  0  0  1  0  0  0  0  1   0      2
20    1  1  1  1  1  0  1  0  1   1      7
21    0  0  1  1  1  0  1  0  1   1      6
22    1  1 

Aplica el algoritmo al juego de datos generado

In [66]:
plan, coste = resolver(df2)

print(f'El coste final es {coste} y la planificación es:\n')
plan

El coste final es 48 y la planificación es:



,T1,T2,T3,T4,T5,T6
Día 1,16,29,1,30,28,23
Día 2,20,21,18,8,19,27
Día 3,10,13,12,5,24,17
Día 4,9,25,26,4,3,2
Día 5,11,14,7,15,6,22


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

- https://www.geeksforgeeks.org/timsort/ para hallar la complejidad de timsort.

No he utilizado otras referencias adicionales más que lo aprendido en el máster y en el grado en Matemáticas.

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

El algoritmo propuesto es bastante eficiente en términos de rendimiento, pues es rápido y no ocupa excesiva memoria. No obstante, se puede mejorar el valor del coste de actores y disminuirlo aún más. Esto se podría hacer con un algoritmo parecido al anterior pero que en vez de escoger tomas una a una escoger $N$ tomas a la vez y calcular el coste, de tal manera que nos quedemos con la mejor combinación de esas $N$. Tiene el inconveniente que perdería algo de velocidad (habría que realizar más iteraciones) y ocuparía algo más de memoria (para guardar las posibles soluciones).

Mencionar también que el algoritmo sigue funcionando correctamente bajo posible cambios en el tamaño de los datos de entrada, tanto número de actores como número de tomas.

Para una generalización del problema, podríamos también introducir como input de la función el número máximo de tomas que se pueden realizar.

En mi caso estuve un par de tardes resolviendo previamente este mismo problema pero con número de días variables (donde el problema cambia radicalmente). Obtuve resultados ligeramente peores puesto que el algoritmo es más complicado de hacer ya que implementar una correcta regla voraz es más complicado. Puedo proporcionar el código si es necesario.


Gracias!